# 피마 인디언 당뇨병 예측

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
up = files.upload()

Saving pima-indians-diabetes.csv to pima-indians-diabetes (1).csv


- 데이터 전처리

In [9]:
df = pd.read_csv('/content/pima-indians-diabetes.csv', skiprows=9, encoding = 'utf-8')
df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [10]:
df.columns =['P','G','BP','S','I','BMI','D','Age','Class']
df.head()

,P,G,BP,S,I,BMI,D,Age,Class
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [11]:
# 결측치 확인
df.isna().sum()

P        0
G        0
BP       0
S        0
I        0
BMI      0
D        0
Age      0
Class    0
dtype: int64

In [14]:
# X,y를 ndarray로 추출
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [15]:
X.shape, y.shape

((767, 8), (767,))

- Train/Test dataset으로 분리

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    X,y,stratify = y, test_size = 0.2, random_state = 2022
)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((613, 8), (154, 8), (613,), (154,))

In [19]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([400, 213]))

In [20]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([100,  54]))

- Decision Tree Classifier

In [21]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2022)
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [23]:
params = {
    'max_depth': [2,3,4,5,6],
    'min_samples_split':[2,3,4]
}

- GridSearchCV

In [24]:
from sklearn.model_selection import GridSearchCV
grid_dt = GridSearchCV(dtc,params, scoring='accuracy',cv=5)
grid_dt.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [25]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 2}

In [26]:
best_dt = grid_dt.best_estimator_
best_dt.score(X_test,y_test)

0.7142857142857143

- 실제 적용

In [27]:
X_test[0], y_test[0]

(array([  0.   , 135.   ,  94.   ,  46.   , 145.   ,  40.6  ,   0.284,
         26.   ]), 0)

In [29]:
# 오류가 발생함, 파라미터에는 2D array가 와야함
best_dt.predict(X_test[0])

ValueError: ignored

In [31]:
best_dt.predict(X_test)

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0])

In [32]:
X_test.shape, X_test[0].shape

((154, 8), (8,))

In [33]:
best_dt.predict(X_test[0].reshape(1,-1))

array([0])

In [34]:
pred = best_dt.predict(X_test[0].reshape(1,-1))
print('양성' if pred[0] == 1 else '음성')

음성
